General Usage and Performance Tips
==================================



In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa

## Avoid dtype=object



### How to do it



In [2]:
ser_obj = pd.Series(["foo", "bar", "baz"] * 10_000, dtype=object)
ser_str = pd.Series(["foo", "bar", "baz"] * 10_000, dtype=pd.StringDtype())

In [3]:
ser_str.iloc[0] = False

TypeError: Cannot set non-string value 'False' into a StringArray.

In [4]:
ser_obj.iloc[0] = False

In [5]:
ser_obj.str.capitalize().head()

0    NaN
1    Bar
2    Baz
3    Foo
4    Bar
dtype: object

In [6]:
ser_obj = pd.Series(["foo", "bar", "baz"] * 10_000, dtype=object)
ser_str = pd.Series(["foo", "bar", "baz"] * 10_000, dtype=pd.StringDtype())

In [7]:
import timeit
timeit.timeit(ser_obj.str.upper, number=1000)

2.867934914997022

In [8]:
timeit.timeit(ser_str.str.upper, number=1000)

2.9097740370052634

In [9]:
import io
data = io.StringIO("int_col,string_col\n0,foo\n1,bar\n2,baz")

data.seek(0)
pd.read_csv(data, dtype_backend="numpy_nullable").dtypes

int_col                Int64
string_col    string[python]
dtype: object

In [10]:
df = pd.DataFrame([
    [0, "foo"],
    [1, "bar"],
    [2, "baz"],
], columns=["int_col", "string_col"])
df.convert_dtypes(dtype_backend="numpy_nullable").dtypes

int_col                Int64
string_col    string[python]
dtype: object

In [11]:
import datetime

ser = pd.Series([
    datetime.date(2024, 1, 1),
    datetime.date(2024, 1, 2),
    datetime.date(2024, 1, 3),
])

ser

0    2024-01-01
1    2024-01-02
2    2024-01-03
dtype: object

In [12]:
ser.dt.year

AttributeError: Can only use .dt accessor with datetimelike values

In [13]:
import datetime

ser = pd.Series([
    datetime.date(2024, 1, 1),
    datetime.date(2024, 1, 2),
    datetime.date(2024, 1, 3),
], dtype=pd.ArrowDtype(pa.date32()))

ser

0    2024-01-01
1    2024-01-02
2    2024-01-03
dtype: date32[day][pyarrow]

In [14]:
ser.dt.year

0    2024
1    2024
2    2024
dtype: int64[pyarrow]

## Be cognizant of data sizes



### How to do it



In [15]:
df = pd.DataFrame({
    "a": [0] * 100_000,
    "b": [2 ** 8] * 100_000,
    "c": [2 ** 16] * 100_000,
    "d": [2 ** 32] * 100_000,
})
df = df.convert_dtypes(dtype_backend="numpy_nullable")

df.head()

,a,b,c,d
0,0,256,65536,4294967296
1,0,256,65536,4294967296
2,0,256,65536,4294967296
3,0,256,65536,4294967296
4,0,256,65536,4294967296


In [16]:
df.memory_usage()

Index       128
a        900000
b        900000
c        900000
d        900000
dtype: int64

In [17]:
df.assign(
    a=lambda x: x["a"].astype(pd.Int8Dtype()),
    b=lambda x: x["b"].astype(pd.Int16Dtype()),
    c=lambda x: x["c"].astype(pd.Int32Dtype()),
).memory_usage()

Index       128
a        200000
b        300000
c        500000
d        900000
dtype: int64

In [18]:
df.select_dtypes("number").assign(
    **{x: pd.to_numeric(
         y, downcast="signed", dtype_backend="numpy_nullable"
    ) for x, y in df.items()}
).memory_usage()

Index       128
a        200000
b        300000
c        500000
d        900000
dtype: int64

## Use vectorized functions instead of loops



### How to do it



In [19]:
ser = pd.Series(range(100_000), dtype=pd.Int64Dtype())

In [20]:
ser.sum()

4999950000

In [21]:
result = 0
for x in ser:
    result += x

result

4999950000

In [22]:
timeit.timeit(ser.sum, number=1000)

0.06553813299979083

In [23]:
def loop_sum():
    result = 0
    for x in ser:
        result += x

timeit.timeit(loop_sum, number=1000)

5.465586415004509

In [24]:
df = pd.DataFrame({
    "column": ["a", "a", "b", "a", "b"],
    "value": [0, 1, 2, 4, 8],
})
df = df.convert_dtypes(dtype_backend="numpy_nullable")

for label, group in df.groupby("column"):
    print(f"The group for label {label} is:\n{group}\n")

The group for label a is:
  column  value
0      a      0
1      a      1
3      a      4

The group for label b is:
  column  value
2      b      2
4      b      8



## Avoid mutating data



### How to do it



In [25]:
def mutate_after():
    data = ["foo", "bar", "baz"]
    ser = pd.Series(data, dtype=pd.StringDtype())
    ser.iloc[1] = "BAR"

timeit.timeit(mutate_after, number=1000)

0.0579534339995007

In [26]:
def mutate_before():
    data = ["foo", "bar", "baz"]
    data[1] = "BAR"
    ser = pd.Series(data, dtype=pd.StringDtype())

timeit.timeit(mutate_before, number=1000)

0.03185594100068556

### There's more&#x2026;



## Dictionary-encode low cardinality data



### How to do it



In [27]:
values = ["foo", "bar", "baz"]
ser = pd.Series(values * 100_000, dtype=pd.StringDtype())
ser.memory_usage()

2400128

In [28]:
cat = pd.CategoricalDtype(values)
ser = pd.Series(values * 100_000, dtype=cat)
ser.memory_usage()

300260

## Test-driven development features



### How it works



In [29]:
import unittest

class MyTests(unittest.TestCase):

    def test_42(self):
        self.assertEqual(21 * 2, 42)

def suite():
    suite = unittest.TestSuite()
    suite.addTest(MyTests("test_42"))
    return suite

runner = unittest.TextTestRunner()
runner.run(suite())

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [30]:
def some_cool_numbers():
    return pd.Series([42, 555, pd.NA], dtype=pd.Int64Dtype())

class MyTests(unittest.TestCase):

    def test_cool_numbers(self):
        result = some_cool_numbers()
        expected = pd.Series([42, 555, pd.NA], dtype=pd.Int64Dtype())
        self.assertEqual(result, expected)

def suite():
    suite = unittest.TestSuite()
    suite.addTest(MyTests("test_cool_numbers"))
    return suite

runner = unittest.TextTestRunner()
runner.run(suite())

E
ERROR: test_cool_numbers (__main__.MyTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_172691/2361126771.py", line 9, in test_cool_numbers
    self.assertEqual(result, expected)
  File "/usr/lib/python3.9/unittest/case.py", line 837, in assertEqual
    assertion_func(first, second, msg=msg)
  File "/usr/lib/python3.9/unittest/case.py", line 827, in _baseAssertEqual
    if not first == second:
  File "/home/willayd/clones/Pandas-Cookbook-Third-Edition/lib/python3.9/site-packages/pandas/core/generic.py", line 1577, in __nonzero__
    raise ValueError(
ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

----------------------------------------------------------------------
Ran 1 test in 0.004s

FAILED (errors=1)


<unittest.runner.TextTestResult run=1 errors=1 failures=0>

In [31]:
result = some_cool_numbers()
expected = pd.Series([42, 555, pd.NA], dtype=pd.Int64Dtype())
result == expected

0    True
1    True
2    <NA>
dtype: boolean

In [32]:
import pandas.testing as tm

def some_cool_numbers():
    return pd.Series([42, 555, pd.NA], dtype=pd.Int64Dtype())

class MyTests(unittest.TestCase):

    def test_cool_numbers(self):
        result = some_cool_numbers()
        expected = pd.Series([42, 555, pd.NA], dtype=pd.Int64Dtype())
        tm.assert_series_equal(result, expected)

def suite():
    suite = unittest.TestSuite()
    suite.addTest(MyTests("test_cool_numbers"))
    return suite

runner = unittest.TextTestRunner()
runner.run(suite())

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [33]:
def some_cool_numbers():
    return pd.Series([42, 555, pd.NA], dtype=pd.Int64Dtype())

class MyTests(unittest.TestCase):

    def test_cool_numbers(self):
        result = some_cool_numbers()
        expected = pd.Series([42, 555, pd.NA], dtype=pd.Int32Dtype())
        tm.assert_series_equal(result, expected)

def suite():
    suite = unittest.TestSuite()
    suite.addTest(MyTests("test_cool_numbers"))
    return suite

runner = unittest.TextTestRunner()
runner.run(suite())

F
FAIL: test_cool_numbers (__main__.MyTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_172691/2197259517.py", line 9, in test_cool_numbers
    tm.assert_series_equal(result, expected)
  File "/home/willayd/clones/Pandas-Cookbook-Third-Edition/lib/python3.9/site-packages/pandas/_testing/asserters.py", line 975, in assert_series_equal
    assert_attr_equal("dtype", left, right, obj=f"Attributes of {obj}")
  File "/home/willayd/clones/Pandas-Cookbook-Third-Edition/lib/python3.9/site-packages/pandas/_testing/asserters.py", line 421, in assert_attr_equal
    raise_assert_detail(obj, msg, left_attr, right_attr)
  File "/home/willayd/clones/Pandas-Cookbook-Third-Edition/lib/python3.9/site-packages/pandas/_testing/asserters.py", line 614, in raise_assert_detail
    raise AssertionError(msg)
AssertionError: Attributes of Series are different

Attribute "dtype" are different
[left]:  Int64
[right]: Int32

----

<unittest.runner.TextTestResult run=1 errors=0 failures=1>

### There's more&#x2026;

